In [46]:
import requests
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import config
import logging
import importlib
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import validators
import pickle
import time
import random
from calendar import monthrange
import re
import spacy
from spacy.tokens import Doc, Span
import nltk
from nltk.corpus import stopwords
from nltk.corpus import treebank
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.corpus import wordnet

from nltk import pos_tag, sent_tokenize, wordpunct_tokenize

pd.options.display.max_columns = 300
import string
import unicodedata

pd.options.display.max_columns = 200
pd.options.display.max_rows = 999
import matplotlib.pyplot as plt  

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets as datasets
import pandas as pd
import numpy as np
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.metrics import mean_squared_error
from pandas.plotting import scatter_matrix
import seaborn as sns
import scipy.stats as stats
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.dummy import DummyClassifier
from sklearn.externals import joblib
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn import tree 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.externals.six import StringIO  
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.svm import SVC  
from time import time
np.random.seed(0)
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline


import gensim
from gensim import corpora
from gensim import models
from gensim.sklearn_api import lsimodel, ldamodel
from gensim.matutils import sparse2full
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel

import pyLDAvis
import pyLDAvis.gensim

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from functools import partial
from operator import itemgetter

from joblib import dump, load


In [84]:
class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, extra_stop_words, bigram_list, language='english', source=None):
        self.source = source
        self.stopwords = set(nltk.corpus.stopwords.words(language))
        self.more_stops = extra_stop_words
        self.stopwords.update(self.more_stops) 
        self.lemmatizer = WordNetLemmatizer()
        
        self.bigram_list = bigram_list        
        self.first_grams = [word.split()[0] for word in bigram_list]
        self.second_grams = [word.split()[1] for word in bigram_list]

    def get_custom_bigrams(self, tokens):
        new_toks = []
        length = len(tokens)

        for ix, tok in enumerate(tokens):
            try:
                if tok in self.first_grams:
                    if ix+1 <= length and tokens[ix+1] in self.second_grams:
                        new_toks.append(' '.join([tok, tokens[ix+1]]))
                        continue
                if tok in self.second_grams:
                    # we assume if we see the 2nd token in a bigram that
                    # we've already prcessed the first and second together
                    # so skip 
                    continue
                new_toks.append(tok)
            except IndexError:
                continue

        return new_toks        

    def is_punct(self, token):
        return all(unicodedata.category(char).startswith('P') for char in token)

    def is_stopword(self, token):
        return token.lower() in self.stopwords
    
    def get_stopwords(self):
        return self.stopwords

    def get_wordnet_pos(selfm, word):
        #Map POS tag to first character lemmatize() 
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}

        return tag_dict.get(tag, wordnet.NOUN)

    def normalize(self, document):
        NYT_PAT = '^[A-Z]* — '
        NYT_COMP_PAT = re.compile(NYT_PAT)        
        
        norm_toks = []
    
        if self.source and self.source == 'NYT':
            document = NYT_COMP_PAT.sub('', document)
        
        if self.source and self.source == 'RT':
            document = self.rt_clean(document)
            
        # NOTE:  I found nltk's sentenizer to perform poorly
            #        for sent in sent_tokenize(document):
        for sent in document.split('.'):            
            for tok in nltk.word_tokenize(sent):
                if self.is_punct(tok) or tok.isdigit():
                    continue
                lem = self.lemmatize(tok, self.get_wordnet_pos(tok)).lower()
                if not self.is_stopword(lem):
                    norm_toks.append(lem) 
#         bigram_toks = self.get_custom_bigrams(norm_toks)
#         return ' '.join(bigram_toks)
#         return ' '.join(norm_toks)       
        return ' '.join(self.get_custom_bigrams(norm_toks))
    
    def rt_clean(self, text):
        #
        # first match this simple, typical Reuters pattern:
        # "(Reuters) - British Prime Minister Boris Johnson..."
        rt_pat = r'^\(Reuters\) - '
        RT_PAT = re.compile(rt_pat)
        text = RT_PAT.sub('', text)

        rt_pat2 = r'^\(This.*\)'

        RT_PAT2 = re.compile(rt_pat2)
        text = RT_PAT2.sub('', text)

        # match this:  "ANKARA/Turkish-backed Syrian...""
        # or this:  "MAMALLAPURAM, India, Chinese President...""
        # or this:  "ADDIS For Samson Berhane, the news that...""
        caps_loc_pat = r'[A-Z]{2,20}[ \/,]{1}'
        CAPS_PAT = re.compile(caps_loc_pat)

        # After removing first pattern, 
        # match this:  "IndiaAt least five people were injured in a grenade attack"
        # but save "At least..."
        name_against_text_pat = r'^ *[A-Z][a-z]{2,20}([A-Z]{1})'
        NM_TEXT_PAT = re.compile(name_against_text_pat)

        first_sub = CAPS_PAT.sub('', text)

        match = NM_TEXT_PAT.match(first_sub)
        rep_char = ''
        if match:
            rep_char = match[1]

        t = NM_TEXT_PAT.sub('', first_sub)
        # now add back that lone capital letter
        text = rep_char + t

        return text    


    def lemmatize(self, token, pos):
        return self.lemmatizer.lemmatize(token, pos)

    def fit(self, X, y=None):
        self.source = y
        return self

    def transform(self, documents):
        for doc in documents:
            yield self.normalize(doc)

In [36]:
# code to pull out our custom bi-grams
bigrams = ['donald trump', 'angela merkel', 'boris johnson', 'vladimir putin', 'benjamin netanyahu', 
           'hong kong', 'north korea', 'south korea', 'united kingdom', 'united states', 'south africa', 
           'xi jinping', 'carrie lam', 'oleksiy honcharuk', 'volodymyr zelensky', 'emmanuel macron',
           'viktor orbán', 'justin trudeau', 'north america', 'south america', 'sinn fein', 'jeremy corbyn', 'narendra modi',
            'mohamed morsi', 'shuping wang', 'hassan rouhani', 'rudy giuliani', 'joe biden', 'new zealand', 'european union', 'pope francis']

In [37]:
more_stops = {'mr', 'ms', 'mr.', 'ms.', 'mrs', 'mrs.','say', 'said', 'saying', 
                           'also', 'yeh', 'hom', 'even', 'like', 'k', 'n', 'u', 'would', 'could', '$'}

## Notes for pipeline automation

1. Fetch article (denote source)
2. Scrape article
3. Run transformation pipeline
   - Use instantiated text normalizer 
   - Use instantiated bigram transformer
   - "stringify" tokens (' '.join())
   - Create PD Series of text data
   - Vectorize text data, based on fitted TD-IDF vector model
   - Pass vector to model.predict()
   - Pull out predicted label
   
 Questions:  should I use a SKLearn Pipeline?

In [85]:
def create_and_run_pipeline(doc, model, vector, extra_stops=more_stops, bigrams=bigrams, source=None):
    textNormer = TextNormalizer(extra_stop_words=more_stops, bigram_list=bigrams, source=source)
    text = textNormer.normalize(doc)
    print(text)
    text_series = pd.Series(text)
    print(text_series)
    text_vec = vector.transform(text_series)
    train_pred = model.predict(text_vec.toarray())
    return train_pred

In [86]:
textNormer = TextNormalizer(extra_stop_words=more_stops, bigram_list=bigrams, source=None)

In [87]:
textNormer.normalize(test_article)

'russian cyber espionage unit hack iranian hacker lead attack country joint uk us investigation reveal so-called turla group link russian intelligence allegedly hijack tool oilrig group widely link iranian government accord two-year probe uk national cyber security centre collaboration us national security agency ncsc part gchq digital intelligence agency iranian group likely unaware hack method hack deployed another cyber espionage team security official involve investigation victims include military establishment government department scientific organisation university across world mainly middle east paul chichester ncsc director operation turla activity represent real change modus operandi cyber actor add sense confusion state-backed cyber group responsible successful attack reason publicise different tradecraft see turla use told reporter want others able understand activity chichester described turla begin piggyback oilrig attack monitoring iranian hack closely enough use backdoor

## Get our persisted model and TF-IDF Vector

In [47]:
tfidf = joblib.load('tfidf.joblib')

In [48]:
model = joblib.load('gnb-model.joblib')

In [43]:
with open('test_article.pkl', 'rb') as f:
    test_article = pickle.load(f)

In [44]:
test_article

'A Russian cyber espionage unit has hacked Iranian hackers to lead attacks in more than 35 countries, a joint UK and US investigation has revealed.\n\nThe so-called Turla group, which has been linked with Russian intelligence, allegedly hijacked the tools of Oilrig, a group widely linked to the Iranian government, according to a two-year probe by the UK’s National Cyber Security Centre in collaboration with the US’ National Security Agency. The NCSC is part of GCHQ, the digital intelligence agency.\n\nThe Iranian group is most likely unaware that its hacking methods have been hacked and deployed by another cyber espionage team, security officials involved in the investigation said. Victims include military establishments, government departments, scientific organisations and universities across the world, mainly in the Middle East.\n\nPaul Chichester, NCSC director of operations, said Turla’s activity represented “a real change in the modus operandi of cyber actors” which he said “added

In [88]:
create_and_run_pipeline(test_article, model, tfidf)

russian cyber espionage unit hack iranian hacker lead attack country joint uk us investigation reveal so-called turla group link russian intelligence allegedly hijack tool oilrig group widely link iranian government accord two-year probe uk national cyber security centre collaboration us national security agency ncsc part gchq digital intelligence agency iranian group likely unaware hack method hack deployed another cyber espionage team security official involve investigation victims include military establishment government department scientific organisation university across world mainly middle east paul chichester ncsc director operation turla activity represent real change modus operandi cyber actor add sense confusion state-backed cyber group responsible successful attack reason publicise different tradecraft see turla use told reporter want others able understand activity chichester described turla begin piggyback oilrig attack monitoring iranian hack closely enough use backdoor 

array(['NYT'], dtype='<U3')

In [42]:
#
# I think we're holding off on using SKLearn's pipeline, due to the necessity to know
# the source of each article for our transformation - alas!
#
# def create_pipeline(estimator, stop_words, bigrams):
#     steps = [
#         ('normalize', TextNormalizer(stop_words, bigrams)),
#         ('vectorize', TfidfVectorizer(stop_words='english', min_df=5, max_df=.5))
#     ]
#     steps.append(('classifier', estimator))
#     return Pipeline(steps)

In [18]:
with open('news_doc_pop_df.pkl', 'rb') as f:
    news_doc_pop_df = pickle.load(f)

In [19]:
news_doc_pop_df.head()

,paper,text
240,NYT,MOSCOW — Allies of President Vladimir V. Putin...
2306,NYT,WASHINGTON — President Trump warned on Tuesday...
2106,RT,FRANKFURT/The German government on Friday pres...
236,FT,"Like many Mexican business people, Armando San..."
333,FT,"Charles Li, chief executive of Hong Kong Excha..."


In [20]:
len(news_doc_pop_df)

6669

In [22]:
X_all = news_doc_pop_df[['text']]

In [23]:
y_all = news_doc_pop_df[['paper']]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.2, random_state = 42)

In [39]:
model = create_pipeline(MultinomialNB(), more_stops, bigrams)

In [40]:
model.fit(X_train, y_train)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().